In [1]:
import torch
import torch.nn.functional as F
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import numpy as np

In [2]:
with open('dict.txt', 'r', encoding='utf-8') as f:
    lines = [line.strip() for line in f.readlines()]

# Вывод словаря с токенами

In [3]:
import random

def generate_typo(word):
    """
    Генерирует опечатку в слове. Типы опечаток: удаление буквы, добавление буквы, перестановка двух букв.
    """
    typo_type = random.choice(['delete', 'add', 'transpose'])
    if typo_type == 'delete' and len(word) > 1:  # Удаление буквы
        pos = random.randint(0, len(word) - 1)
        return word[:pos] + word[pos + 1:]
    elif typo_type == 'add':  # Добавление буквы
        pos = random.randint(0, len(word))
        char = random.choice('абвгдеёжзийклмнопрстуфхцчшщъыьэюя')
        return word[:pos] + char + word[pos:]
    elif typo_type == 'transpose' and len(word) > 1:  # Перестановка двух букв
        pos = random.randint(0, len(word) - 2)
        return word[:pos] + word[pos + 1] + word[pos] + word[pos + 2:]
    else:  # Если слово состоит из одной буквы, возвращаем его без изменений
        return word

# Генерация примера тренировочных данных
training_data = [(word, generate_typo(word)) for word in lines[:1000]]  # Генерируем данные для первых 1000 слов

# Показываем несколько примеров сгенерированных пар
training_data[:10]


[('абажур', 'ажбажур'),
 ('абажурчик', 'абажучрик'),
 ('абазин', 'абазни'),
 ('абазинец', 'абазиняец'),
 ('абазинцы', 'аазинцы'),
 ('абазинка', 'абназинка'),
 ('абазия', 'баазия'),
 ('абандон', 'абандо'),
 ('аббат', 'аббт'),
 ('аббатиса', 'бабатиса')]

In [4]:
class TypoDataset(Dataset):
    def __init__(self, word_list):
        self.word_list = word_list
    
    def __len__(self):
        return len(self.word_list)
    
    def __getitem__(self, idx):
        correct_word, typo_word = self.word_list[idx]
        return typo_word, correct_word

In [5]:
dataset = TypoDataset(training_data)  # Используйте training_data, сгенерированные ранее
data_loader = DataLoader(dataset, batch_size=32, shuffle=True)

In [6]:
class Encoder(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(Encoder, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size)
    
    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output, hidden = self.lstm(embedded, hidden)
        return output, hidden
    
    def initHidden(self):
        return (torch.zeros(1, 1, self.hidden_size),
                torch.zeros(1, 1, self.hidden_size))

In [7]:
class Decoder(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(Decoder, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)
    
    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.lstm(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden
    
    def initHidden(self):
        return (torch.zeros(1, 1, self.hidden_size),
                torch.zeros(1, 1, self.hidden_size))

In [8]:
input_size = 256  # Пример размера входного словаря
output_size = 256  # Пример размера выходного словаря
hidden_size = 512  # Размер скрытого слоя
encoder = Encoder(input_size, hidden_size)
decoder = Decoder(hidden_size, output_size)

# Оптимизатор и функция потерь
optimizer = optim.Adam(list(encoder.parameters()) + list(decoder.parameters()), lr=0.01)
criterion = nn.NLLLoss()

In [9]:
def train(input_tensor, target_tensor, encoder, decoder, optimizer, criterion):
    optimizer.zero_grad()

    encoder_hidden = encoder.initHidden()
    encoder_outputs = torch.zeros(encoder.hidden_size)

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = target_tensor[0]  # Начинаем с первого символа

    decoder_hidden = encoder_hidden

    for di in range(1, target_length):
        decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
        loss += criterion(decoder_output, target_tensor[di].unsqueeze(0))
        decoder_input = target_tensor[di]  # Используем правильный следующий символ для декодирования

    loss.backward()
    optimizer.step()

    return loss.item() / target_length

In [10]:
for epoch in range(1):  # Пример с одной эпохой для демонстрации
    for input_tensor, target_tensor in data_loader:
        # Здесь необходимо преобразовать input_tensor и target_tensor к нужному формату
        # Например, используя функцию для преобразования слов в тензоры индексов
        loss = train(input_tensor, target_tensor, encoder, decoder, optimizer, criterion)
        print(f'Loss: {loss}')

AttributeError: 'tuple' object has no attribute 'size'